# Mental Health Analysis 

# ☑️ Setting up the database connection

Run the following code cell to import `pandas` and `sqlite3` libraries and create the connection to the `mental_health.sqlite` database.

**Do not change this code!** The `conn` variable will be used throughout the notebook to query the database.

In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('/Users/fatmagulgok/Desktop/spark/mental_health/mental_health.sqlite')

# ☑️ Introduction to the Mental Health dataset 



The database contains three tables: `Survey`, `Question`, and `Answer`.

  1. **Survey**, containing columns:
    - `PRIMARY KEY INT SurveyID`
    - `TEXT Description`


  2. **Question**, containing columns: 
    - `PRIMARY KEY QuestionID`
    - `TEXT QuestionText`


  3. **Answer**, containing columns:
    - `PRIMARY/FOREIGN KEY SurveyID`
    - `PRIMARY KEY UserID`
    - `PRIMARY/FOREIGN KEY QuestionID`
    - `TEXT AnswerText`


`SuveyID` column contains the survey year i.e. 2014, 2016, 2017, 2018, 2019 and the same question can be used for multiple surveys. 

Some questions can contain multiple answers, thus the same user can appear more than once for any given QuestionID.

Run the following code cell to show all the tables in the `mental_health.sqlite` database:

In [2]:
query = """
SELECT name 
FROM sqlite_master 
WHERE type='table';
"""
df = pd.read_sql_query(query, conn)
df

,name
0,Answer
1,Question
2,Survey


Referencing these tables and their respective columns will be useful in answering the following questions. Run the following code to show column names and data types within each table:

In [3]:
for table in ['Answer','Question','Survey']:
    
    query = f"""
    PRAGMA table_info({table});
    """
    df = pd.read_sql_query(query, conn)
    print(df[['name','type']])
    print('='*40)

         name            type
0  AnswerText  VARCHAR(10000)
1    SurveyID         INTEGER
2      UserID         INTEGER
3  QuestionID         INTEGER
           name           type
0  questiontext  VARCHAR(1000)
1    questionid        INTEGER
          name          type
0     SurveyID       INTEGER
1  Description  VARCHAR(255)


# ☑️ Queries

**1. Write a SQL query that finds all the records within the `Question` table where the `questionid` is equal to 2 or 3. The columns should be called `Question` and `ID`**

```

In [4]:
#add your code below
question_2_3_query = """
SELECT questiontext AS Question, questionid AS ID
FROM Question
WHERE questionid = 2 OR questionid = 3
;
"""

question_2_3_result = pd.read_sql(question_2_3_query, conn)
question_2_3_result



,Question,ID
0,What is your gender?,2
1,What country do you live in?,3


**2. Refer to the `Survey` table. Write a SQL query to retrieve the surveys from 2014 and 2017. The columns should be called `Year` and `Year_Description`**

```

In [5]:
#add your code below
survey_years_query = """
SELECT SurveyID AS Year, Description AS Year_Description
FROM Survey
WHERE SurveyID IN (2014, 2017)
;
"""

survey_years_result = pd.read_sql(survey_years_query, conn)
survey_years_result

,Year,Year_Description
0,2014,mental health survey for 2014
1,2017,mental health survey for 2017


**3. Refer to the `AnswerText` column in `Answer` table. Write a SQL query to find out how many answers in total have been given throughout the years. Your result should contain one column, called `answers_count`**

```

In [6]:
#add your code below
number_of_answers_query = """
SELECT COUNT(AnswerText) AS answers_count
FROM Answer
;
"""

number_of_answers_result = pd.read_sql(number_of_answers_query, conn)
number_of_answers_result

,answers_count
0,236898


**4. Refer to the `AnswerText` column in `Answer` table. Write a SQL query to find out how many answers have been given in 2017 and 2019. Your result should contain one column, called `answers_count`**

```

In [7]:
#add your code below
number_of_answers_17_19_query = """
SELECT COUNT(AnswerText) AS answers_count
FROM Answer
WHERE SurveyID IN (2017, 2019)
;
"""

number_of_answers_17_19_result = pd.read_sql(number_of_answers_17_19_query, conn)
number_of_answers_17_19_result

,answers_count
0,84208


**5. Refer to the `AnswerText` column in `Answer` table. Write a SQL query to extract the first 100 answers for the year 2014. Your result should contain one column (the `AnswerText`)**

```

In [8]:
#add your code below
answer_2014_query = """
SELECT AnswerText
FROM Answer
WHERE SurveyID = 2014
LIMIT 100
;
"""

answer_2014_result = pd.read_sql(answer_2014_query, conn)
answer_2014_result

,AnswerText
0,37
1,44
2,32
3,31
4,31
...,...
95,29
96,24
97,31
98,33


**6. Refer to the `Answer` table. For each year of the survey, how many questions have been asked? Return a table containing the survey year and the number of unique questions that have been asked for each year. Call the survey year column `year` and the second column `survey_answers`**

```

In [9]:
#add your code below
answer_per_survey_query = """
SELECT SurveyID AS year, COUNT(DISTINCT(QuestionID)) survey_answers
FROM Answer
GROUP BY SurveyID
;
"""

answer_per_survey_result = pd.read_sql(answer_per_survey_query, conn)
answer_per_survey_result

,year,survey_answers
0,2014,26
1,2016,60
2,2017,76
3,2018,76
4,2019,76


**7. Refer to the `Answer` table. Select the maximum age of the participants for each survey year. Return a table containing the survey year and the maximum age of participants for that year. Your result should contain two columns: one called `year` and one called `max_age`**

- Look at the Question table first to find which question asks participants about their age

```

In [ ]:
xquery = """
SELECT *
FROM Question
LIMIT 3
;
"""

x = pd.read_sql(xquery, conn)
x

In [ ]:
#add your code below
max_age_query = """
SELECT SurveyID AS year, MAX(CAST(AnswerText AS int)) AS max_age
FROM Answer
WHERE QuestionID = 1
GROUP BY SurveyID
;
"""

max_age_result = pd.read_sql(max_age_query, conn)
max_age_result

**8. Refer to the `Answer` table. Write a SQL query that finds out how many people always, never, or sometimes work remotely. Your result should have one column called `answer`, and one called `count`**

- Have a look at the Question table first to find which question asks participants about how often they work remotely. Note that always, never, and sometimes are the three possible answers.

```

In [ ]:
xquery = """
SELECT *
FROM Question
WHERE questiontext LIKE '%remote%'
;
"""

x = pd.read_sql(xquery, conn)
x

In [ ]:
xquery = """
SELECT DISTINCT(AnswerText)
FROM Answer
WHERE QuestionID IN (93, 118)
;
"""

x = pd.read_sql(xquery, conn)
x

In [ ]:
#add your code below
work_remotely_query = """
SELECT AnswerText AS answer, COUNT(AnswerText) AS count
FROM Answer
WHERE QuestionID = 118
GROUP BY AnswerText
;
"""

work_remotely_result = pd.read_sql(work_remotely_query, conn)
work_remotely_result

**9. Refer to the `Answer` table. Write a SQL query that returns the given age of 2016 survey participants as well as the count of participants for each age. Call the age column `participant_age` and the count column `number_of_participants`**

```

In [ ]:
xquery = """
SELECT *
FROM Question
WHERE questiontext LIKE '%age%'
;
"""

x = pd.read_sql(xquery, conn)
x

In [ ]:
#add your code below
age_freq_query = """
SELECT DISTINCT(AnswerText) as participant_age, COUNT(AnswerText) AS number_of_participants
FROM Answer
WHERE SurveyID = 2016 AND QuestionID = 1
GROUP BY participant_age
;
"""

age_freq_result = pd.read_sql(age_freq_query, conn)
age_freq_result

**10. This question refers to the query you wrote in Question 6. Now let's make Question 6 a little bit more complicated and order the year in descending order. Call the survey year column `year` and the count column `survey_answers`**

```

In [ ]:
#add your code below
answer_per_survey_advanced_query = """
SELECT SurveyID AS year, COUNT(DISTINCT(QuestionID)) as survey_answers
FROM Answer
GROUP BY SurveyID
ORDER BY survey_answers DESC, year DESC
;
"""

answer_per_survey_advanced_result = pd.read_sql(answer_per_survey_advanced_query, conn)
answer_per_survey_advanced_result